# Kubeflow Pipelines Getting Started

In [ ]:
!pip install --quiet google-cloud-aiplatform kfp google-cloud-pipeline-components

In [ ]:
from typing import Dict

import kfp
from kfp import dsl, compiler
from kfp.dsl import Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics, Metrics, component

from google.cloud import aiplatform  # For PipelineJob (Vertex AI Pipelines) submission

## Bucket and Constants

In [ ]:
!gsutil mb gs://YOUR_BUCKET

In [ ]:
REGION="us-central1"
PIPELINE_ROOT="gs://YOUR_BUCKET"

In [ ]:
@component(
    base_image="python:3.11",
    output_component_file="first-component.yaml",
)
def product_name(text: str) -> str:
    return text

In [ ]:
@component(
    base_image="python:3.11",
    output_component_file="second-component.yaml",
    packages_to_install=["emoji"]
)
def emoji(text: str) -> Dict[str, str]:
    import emoji
    
    emoji_text = text
    emoji = emoji.emojize(':' + emoji_text + ':', language='alias')
    print("output one: {}; output two: {}".format(emoji_text, emoji))
    
    return {
        "emoji_text": emoji_text,
        "emoji": emoji,
    }

In [ ]:
@component(
    base_image="python:3.11",
    output_component_file="third-component.yaml"
)
def build_sentence(
    product: str,
    emoji: Dict[str, str]
) -> str:
    print("We completed the pipeline!")
    end_str = product + " is "
    if len(emoji["emoji"]) > 0:
        end_str += emoji["emoji"]
    else:
        end_str += emoji["emoji_text"]
    return end_str

In [ ]:
@dsl.pipeline(
    name="hello-world",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def intro_pipeline(
    text: str = "Vertex AI Pipelines",
    emoji_str: str = "sparkles"
):
    product_task = product_name(text=text)
    emoji_task = emoji(text=emoji_str)
    consumer_task = build_sentence(
        product=product_task.output,
        emoji=emoji_task.output,
    )

In [ ]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline,
    package_path="intro_pipeline_job.json"
)

In [ ]:
aiplatform.init()

In [ ]:
job = aiplatform.PipelineJob(
    display_name="Demonstration",
    template_path="intro_pipeline_job.json",
    pipeline_root=PIPELINE_ROOT
)

In [ ]:
job.submit()